# Import Dependencies

In [1]:
#Import dependencies
import pandas as pd
import json

from sqlalchemy import create_engine

import datetime as dt

# Import JSON Files

In [2]:
#Data for rates 2007-05-01 to 2017-04-28 
data = json.load(open('Resources/2017_LEGACY_CLOSING_RATES.json'))
df_2017_currencies = pd.DataFrame(data["seriesDetail"])
#df_2017_currencies.head()

df_2017_observations = pd.DataFrame(data["observations"])
#df_2017_observations.head()



#Data for rates > 2017-04-28 in a different file
data = json.load(open('Resources/FXUSDCAD-sd-2017-05-01-ed-2019-06-28.json'))

df_2018_observations = pd.DataFrame(data["observations"])
#df_2018_observations.head()

# Transform
### Data made available in 2 different files, one with rates for 2007-05-01 to 2017-04-28 & the other for rates after 2017-04-28. 

In [3]:
#Transform 2007-05-01 to 2017-04-28  data
#IEXE0102 - US Closing price
arr = []
for row in range(len(df_2017_observations)):
    arr.append([df_2017_observations['d'][row], df_2017_observations['IEXE0102'][row]['v'] ])
    
df_2017 = pd.DataFrame(arr)

df_2017.rename(columns={0:'Date', 1:'Closing_Rate'}, inplace=True)


df_2017.head() 

,Date,Closing_Rate
0,2007-05-01,1.1105
1,2007-05-02,1.1087
2,2007-05-03,1.1066
3,2007-05-04,1.1075
4,2007-05-07,1.1018


In [4]:
#Transform > 2017-04-28  data

arr = []
for row in range(len(df_2018_observations)):
    arr.append([df_2018_observations['d'][row], df_2018_observations['FXUSDCAD'][row]['v'] ])
    
df_2018 = pd.DataFrame(arr)

df_2018.rename(columns={0:'Date', 1:'Closing_Rate'}, inplace=True)


df_2018.head() 

,Date,Closing_Rate
0,2017-05-01,1.3660
1,2017-05-02,1.3720
2,2017-05-03,1.3714
3,2017-05-04,1.3743
4,2017-05-05,1.3712


# Load Data

In [5]:
#Merge the 2 dataframes
df_2017 = df_2017.append(df_2018)

fx_rates = df_2017.copy()

print(f'Min Date: {fx_rates.Date.min()}, Closing Rate: {fx_rates.Closing_Rate.min()}')
print(f'Max Date: {fx_rates.Date.max()}, Closing Rate: {fx_rates.Closing_Rate.min()}')

Min Date: 2007-05-01, Closing Rate: 0.9215
Max Date: 2019-06-28, Closing Rate: 0.9215


### Get the average rates per month

In [6]:
fx_rates['Date']= pd.to_datetime(fx_rates['Date']) 
fx_rates['Closing_Rate']= pd.to_numeric(fx_rates['Closing_Rate']) 

In [7]:
fx_rates_avg = pd.DataFrame()
fx_rates["Date"] = fx_rates["Date"].dt.strftime('%b %Y')

fx_rates_avg = fx_rates.groupby('Date', as_index=False).agg({"Closing_Rate": "mean"})

fx_rates_avg.rename(columns={"Closing_Rate":"Average_FX"}, inplace=True)

fx_rates_avg.head()

,Date,Average_FX
0,Apr 2008,1.013105
1,Apr 2009,1.224595
2,Apr 2010,1.005052
3,Apr 2011,0.957795
4,Apr 2012,0.992650


In [8]:
rds_connection_string = "postgres: @localhost:5432/border_db"
#rds_connection_string = "postgres: :5432/border_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
fx_rates_avg.to_sql(name='fx_rates', con=engine, if_exists='append', index=False)

In [10]:
pd.read_sql_query('select * from fx_rates', con=engine).head()

,id,Date,Average_FX
0,1,Apr 2008,1.0131
1,2,Apr 2009,1.2246
2,3,Apr 2010,1.0051
3,4,Apr 2011,0.9578
4,5,Apr 2012,0.9927
